In [ ]:
import pandas as pd
import math
import os
import pyperclip
import spotipy
import matplotlib.colors as mcolors
import openpyxl
import numpy as np

from IPython.display import clear_output
from spotipy.oauth2 import SpotifyOAuth
from dotenv import load_dotenv
from openpyxl.styles import PatternFill
from openpyxl.styles.differential import DifferentialStyle
from openpyxl.formatting import Rule
from openpyxl.utils.dataframe import dataframe_to_rows

pd.options.display.float_format = '{:,.0f}'.format

load_dotenv()

In [ ]:
scope = "user-library-read"

sp = spotipy.Spotify(auth_manager = SpotifyOAuth(scope = scope,
                                                 client_id = os.environ.get("spotifyClientId"),
                                                 client_secret = os.environ.get("spotifyClientSecret"),
                                                 redirect_uri = 'https://example.com/callback'))

In [ ]:
taylor_swift = sp.artist('06HL4z0CvFAxyc27GXpf02')

In [ ]:
def get_artist_songs(artist):

    artist_uri = artist['uri']

    # Get artist's albums
    albums = []
    results = sp.artist_albums(artist_uri, album_type='album')
    albums.extend(results['items'])
    while results['next']:
        results = sp.next(results)
        albums.extend(results['items'])

    # Get tracks from albums
    tracks = []
    track_df = pd.DataFrame()
    global album
    for album in albums:
        results = sp.album_tracks(album['id'])
        #tracks.extend(results['items'])
        global album_tracks
        album_tracks = results['items']
        for track in album_tracks:
            track_info = {
                'Track Name': track['name'],
                'Album': album['name'],
                'Release Date': album['release_date'],
                'Duration (ms)': track['duration_ms'],
                'Album Genre': }
            idx = len(track_df)
            cols = list(track_info.keys())
            track_df.loc[idx, cols] = track_info.values()

    return track_df
    


In [ ]:
df = get_artist_songs(taylor_swift)

# Get Lyrics

In [ ]:
import requests
import pandas as pd

# Code and song list from ChatGPT

def fetch_lyrics(artist, title):
    url = f"https://api.lyrics.ovh/v1/{artist}/{title}"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raises an HTTPError if the response status code is 4XX/5XX
        lyrics_data = response.json()
        lyrics = lyrics_data.get('lyrics', '')
        # Remove the initial line if it exists
        if lyrics.startswith("Paroles de la chanson"):
            lyrics = '\n'.join(lyrics.split('\n')[1:])  # Skip the first line
        return lyrics
    except requests.RequestException as e:
        print(f"Error fetching lyrics for {artist} - {title}: {e}")
        return ''

In [ ]:
xx = pd.DataFrame(lyrics_list)

In [ ]:
xx.columns = ['Artist', 'Track Name', 'Lyrics']
xx.drop_duplicates("Track Name", keep = 'first', inplace = True)

In [ ]:
df2 = df.merge(xx[['Track Name', 'Lyrics']],
               on = 'Track Name', how = 'left')

In [ ]:
# ChatGPT defined categories for ~100 songs

love_and_relationships = [
    "Starlight (Taylor's Version)",
    "Back To December",
    "Hey Stephen",
    "White Horse",
    "Love Story",
    "Cruel Summer",
    "The Way I Loved You (Taylor’s Version)",
    "King Of My Heart",
    "Ours (Taylor’s Version)",
    "Tim McGraw",
    "Our Song",
    "Our Song",
    "This Love",
    "All You Had To Do Was Stay",
    "Everything Has Changed (feat. Ed Sheeran) (Taylor’s Version)",
    "Paper Rings",
    "We Are Never Ever Getting Back Together",
    "I Almost Do"
]

self_discovery_and_empowerment = [
    "Shake It Off (Taylor's Version)",
    "Welcome To New York",
    "Vigilante Shit",
    "...Ready For It?",
    "How You Get The Girl",
    "Bad Blood",
    "We Are Never Ever Getting Back Together (Taylor's Version)",
    "22 (Taylor's Version)",
    "New Romantics (Taylor's Version)",
    "Karma",
    "Bad Blood (Taylor's Version)",
    "Mr. Perfectly Fine (Taylor’s Version) (From The Vault)"
]

storytelling_and_reflection = [
    "marjorie",
    "The Lucky One",
    "Fearless",
    "illicit affairs",
    "tolerate it",
    "State Of Grace",
    "Dear John (Taylor's Version)",
    "The Best Day",
    "Maroon",
    "Anti-Hero",
    "The Archer",
    "Never Grow Up",
    "seven",
    "seven",
    "Holy Ground",
    "evermore (feat. Bon Iver)",
    "the last great american dynasty",
    "mirrorball",
    "The Best Day",
    "august",
    "Haunted",
    "ivy",
    "Dear Reader",
    "Tell Me Why (Taylor’s Version)",
    "Sparks Fly"
]

other = [
    "That’s When (feat. Keith Urban) (Taylor’s Version) (From The Vault)",
    "Bigger Than The Whole Sky",
    "Paris",
    "Midnight Rain",
    "Come In With The Rain (Taylor’s Version)",
    "Dancing With Our Hands Tied",
    "Breathe",
    "Breathe",
    "I Know Places",
    "Wildest Dreams (Taylor's Version)",
    "willow",
    "Foolish One (Taylor’s Version) (From The Vault)",
    "Snow On The Beach (feat. More Lana Del Rey)",
    "Wonderland",
    "The Story Of Us",
    "Bejeweled",
    "Untouchable (Taylor’s Version)",
    "Labyrinth",
    "Innocent",
    "Come Back...Be Here",
    "Sad Beautiful Tragic (Taylor's Version)",
    "When Emma Falls in Love (Taylor’s Version) (From The Vault)",
    "High Infidelity",
    "epiphany",
    "Mean",
    "Snow On The Beach (feat. Lana Del Rey)",
    "Jump Then Fall (Taylor’s Version)",
    "Stay Stay Stay (Taylor's Version)",
    '"Slut!" (Taylor\'s Version) (From The Vault)',
    "Fifteen (Taylor’s Version)",
    "Timeless (Taylor’s Version) (From The Vault)",
    "Suburban Legends (Taylor's Version) (From The Vault)",
    "closure"
]

df2['song_theme'] = None
cats = ['other', 'storytelling_and_reflection',
       'self_discovery_and_empowerment', 'love_and_relationships']

for key, row in df2.iterrows():
    song = row['Track Name']
    for l in cats:
        default = df2.loc[key, 'song_theme']
        df2.loc[key, 'song_theme'] = default or (l if song in globals()[l] else None)

In [ ]:
df2.to_clipboard()